In [ ]:
%pip install anthropic


In [ ]:
from anthropic import Anthropic
from helper import load_env


In [ ]:
load_env()

In [ ]:
client = Anthropic()

MODEL_NAME = "claude-3-5-sonnet-20241022"


## Content Blocks

In [ ]:
messages = [
    {
        "role": "user",
        "content": "tell me a joke"
    }
]

response = client.messages.create(
    messages=messages,
    model=MODEL_NAME,
    max_tokens=200,
)

print(response.content[0].text)


In [ ]:
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": "who",
            },
            {
                "type": "image_url",
                "text": "made",
            },
            {
                "type": "text",
                "text": "you?",
            },
        ]
    }
]

response = client.messages.create(
    messages=messages,
    model=MODEL_NAME,
    max_tokens=200
)

print(response.content[0].text)


## Image Prompts


In [ ]:
from IPython.display import Image

Image(filename="images/food.png")


## Image Messages

In [ ]:
import base64

with open("images/food.png") as image_file:
    binary_data = image_file.read()
    base_64_encoded_data = base64.b64encode(binary_data)
    base_64_string = base_64_encoded_data.decode("utf-8")


In [ ]:
base_64_string[:100]


In [ ]:
messages = [
    {
        "role": "user",
        "content": [{
            "type": "image",
            "source": {
                "type": "base64",
                "media_type": "image/png",
                "data": base_64_string
            },
        },{
            "type": "text",
            "text": """How many to-go containers of each type are in this image?"""
        }]
    }
]


In [ ]:
response = client.messages.create(
    messages=messages,
    model=MODEL_NAME,
    max_tokens=200
)

print(response.content[0].text)


# Image Block Helper Function

In [ ]:
import base64
import mimetypes

def create_image_message(image_path):
    with open(image_path, "rb") as image_file:
        binary_data = image_file.read()
    base64_encoded_data = base64.b64encode(binary_data)
    base64_string = base64_encoded_data.decode('utf-8')
    mime_type, _ = mimetypes.guess_type(image_path)
    image_block = {
        "type": "image",
        "source": {
            "type": "base64",
            "media_type": mime_type,
            "data": base64_string
        }
    }
    
    
    return image_block

In [ ]:
Image("images/plant.png")


In [ ]:
messages = [
    {
        "role": "user",
        "content": [
            create_image_message("./images/plant.png"),
            {"type": "text", "text": "What species is this?"}
        ]
    }
]

response = client.messages.create(
    model=MODEL_NAME,
    max_tokens=2048,
    messages=messages
)

print(response.content[0].text)


## A Real World Use Case

In [ ]:
Image(filename='images/invoice.png') 

In [ ]:
messages = [
    {
        "role": "user",
        "content": [
            create_image_message("./images/invoice.png"),
            {"type": "text", "text": """
                Generate a JSON object representing the contents
                of this invoice.  It should include all dates,
                dollar amounts, and addresses. 
                Only respond with the JSON itself.
            """
            }
        ]
    }
]

response = client.messages.create(
    model=MODEL_NAME,
    max_tokens=2048,
    messages=messages
)

print(response.content[0].text)

# Streaming

In [ ]:
response = client.messages.create(
    max_tokens=1024,
    messages=[{"role": "user", "content": "write a poem"}],
    model=MODEL_NAME,
)

print(response.content[0].text)

In [ ]:
with client.messages.stream(
    max_tokens=1024,
    messages=[{"role": "user", "content": "write a poem"}],
    model=MODEL_NAME,
) as stream:
    for text in stream.text_stream:
        print(text, end="", flush=True)

# Computer Use

In [ ]:
def _make_api_tool_result(
        result: ToolResult,
        tool_use_id: str
) -> BetaToolResultBlockParam:
    """Convert an agent ToolResult ot an API ToolResultBlockParam."""
    tool_result_content: list[BetaTextBlockParam | BetaImageBlockParam] | str = []
    is_error = False
    if result.error:
        is_error = True
        tool_result_content = _maybe_prepend_system_tool_result(result, result.error)
    else:
        if result.output:
            tool_result_content.append(
                {
                    "type": "text",
                    "text": _maybe_prepend_system_tool_result(result, result.output)
                }
            )
        if result.base64_image:
            tool_result_content.append(
                {
                    "type": "image",
                    "source": {
                        "type": "base64",
                        "media_type": "image/png",
                        "data": result.base64_image
                    }
                }
            )
    return {
        "type": "tool_result",
        "content": tool_result_content,
        "tool_use_id": tool_use_id,
        "is_error": is_error,
    }

# Real World Prompting